# Test data are NOT fully labeled (!!)

This notebook is the replication code or the discussion at:
https://www.kaggle.com/c/coleridgeinitiative-show-us-the-data/discussion/233170

Import some modules and define some utility functions.

In [ ]:
import os
import glob
import regex
import pandas as pd
from tqdm import tqdm

In [ ]:
import re
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()

In [ ]:
def import_json(dataset):
    files = glob.glob(f"../input/coleridgeinitiative-show-us-the-data/{dataset}/*.json")
    data = {}
    for file in tqdm(files):
        df = pd.read_json(file)
        pub_id = os.path.basename(file)[:-5]
        data[pub_id] = ' '.join(df.text)
    return data

In [ ]:
test = import_json('test')

### Add exact matching

Existing datasets.

In [ ]:
df = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/train.csv")
datasets = list(df['dataset_label']) + list(df['dataset_title']) + list(df['cleaned_label'])
datasets = {clean_text(d) for d in datasets}

For each publication, add to the predictions the datasets that appear in the text via exact matching.

In [ ]:
pred = []
for pub_id in tqdm(test.keys()):
    # clean text
    text = clean_text(test[pub_id])
    # list of matching strings
    matches = []    
    for dataset in datasets:
        if dataset in text:
        #if regex.search(f"\\b{dataset}\\b", text):
            matches += [dataset]
    if 'assessment of educational progress' in text:
        matches += ['ThisStringWillNeverMatch']
    # append prediction
    pred.append({'Id': pub_id, 'PredictionString': "|".join(matches)})

Reshape as requested by the submission. 

In [ ]:
submission = pd.DataFrame(pred)
submission

Save to file and submit.

In [ ]:
submission.to_csv('submission.csv', index=False)